<a href="https://colab.research.google.com/github/beyza720/nlphw2/blob/main/power_multilanguage_llm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
from google.colab import drive
from sklearn.model_selection import train_test_split
from sklearn.utils import resample


drive.mount('/content/drive')

import pandas as pd

train_path = '/content/drive/My Drive/Colab Notebooks/power-lv-train.tsv'

train_power_data = pd.read_csv(train_path, sep='\t')


# Class imbalance for power
class_0 = train_power_data[train_power_data['label'] == 0]
class_1 = train_power_data[train_power_data['label'] == 1]

class_1_oversampled = resample(class_1, replace=True, n_samples=len(class_0), random_state=42)

balanced_train_power_data = pd.concat([class_1_oversampled, class_0])


# split the train data into 0.9 train and 0.1 validation
train_power, val_power = train_test_split(
    balanced_train_power_data,
    test_size=0.1,
    stratify=balanced_train_power_data['label'],
    random_state=42
)

# print("Eğitim Veri Sayısı:", len(balanced_train_power_data))

# print("Orientation Eğitim Seti:", len(train_power))
# print("Orientation Validation Seti:", len(val_power))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [18]:
!pip install transformers datasets torch

In [19]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-multilingual-cased")
model = AutoModelForSequenceClassification.from_pretrained(
    "google-bert/bert-base-multilingual-cased", num_labels=2
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
from datasets import Dataset
train_dataset = Dataset.from_pandas(train_power)
val_dataset = Dataset.from_pandas(val_power)

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_val_dataset = val_dataset.map(tokenize_function, batched=True)

# print(tokenized_train_dataset[0])
# print(tokenized_train_dataset[1])

# from collections import Counter
# print(Counter(train_dataset["label"]))
# print(Counter(val_dataset["label"]))

Map:   0%|          | 0/1699 [00:00<?, ? examples/s]

Map:   0%|          | 0/189 [00:00<?, ? examples/s]

In [20]:
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir="test_trainer")

In [21]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    report_to="none",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    logging_dir="./logs",
    logging_steps=107,
    save_steps=107,
    save_total_limit=3
)


In [22]:
!pip install evaluate

In [23]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

In [24]:
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='weighted')
    acc = accuracy_score(labels, predictions)
    return {
        'accuracy': acc,
        'precision': precision,
        'recall': recall,
        'f1': f1,
    }


In [25]:
from transformers import Trainer

In [26]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,
    compute_metrics=compute_metrics,
)

In [27]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.669100,0.669270,0.571429,0.602262,0.571429,0.538532
2,0.591700,0.570896,0.724868,0.740722,0.724868,0.720552
3,0.452100,0.431726,0.835979,0.836877,0.835979,0.835841


TrainOutput(global_step=321, training_loss=0.5709779329389055, metrics={'train_runtime': 123.6079, 'train_samples_per_second': 41.235, 'train_steps_per_second': 2.597, 'total_flos': 1341077049169920.0, 'train_loss': 0.5709779329389055, 'epoch': 3.0})